In [1]:
# Packages and Imports
import numpy as np
from numpy.random import seed
from numpy.random import rand
from numpy.random import randn
import random
import matplotlib.pyplot as plt

# Accuracy score and confusion matrix
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix 

# Backprop optimizer
from tensorflow.keras.optimizers.legacy import Adam 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense

# Genetic packages
import pygad.kerasga
import pygad
import pygad.nn
import tensorflow.keras

# Quick fix for an issue where my Jupyter Kernel kept dying
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

/Users/freddiejones/miniconda3/envs/tensorflow/lib/python3.10/site-packages/jax/_src/lib/__init__.py:32: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


In [2]:
# Load dataset
wine_data = np.loadtxt('/Users/freddiejones/Desktop/EvolutionaryML/winequality-red.csv', delimiter=',', skiprows=1) # print(wine_data.shape)
np.set_printoptions(formatter={'float': lambda x: '{0:0.2f}'.format(x)})

In [3]:
# Binary classification transformation
wine_data[wine_data[:, -1] < 5.5, -1] = 0 
wine_data[wine_data[:, -1] >= 5.5, -1] = 1
print(wine_data)

[[7.40 0.70 0.00 ... 0.56 9.40 0.00]
 [7.80 0.88 0.00 ... 0.68 9.80 0.00]
 [7.80 0.76 0.04 ... 0.65 9.80 0.00]
 ...
 [6.30 0.51 0.13 ... 0.75 11.00 1.00]
 [5.90 0.65 0.12 ... 0.71 10.20 0.00]
 [6.00 0.31 0.47 ... 0.66 11.00 1.00]]


In [4]:
# Balance dataset
np.random.shuffle(wine_data)

In [5]:
# Split into test and train
thirty_percent = int(0.3 * len(wine_data[:, 0]))

X_test = wine_data[:thirty_percent, :-1] 
Y_test = wine_data[:thirty_percent, -1]

X_train = wine_data[thirty_percent:, 0:-1] 
Y_train = wine_data[thirty_percent:, -1]

In [6]:
# Normalize wine data
X_train = (X_train - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)
X_test = (X_test - np.mean(X_test, axis=0)) / np.std(X_test, axis=0)

In [7]:
np.random.seed(0)

In [8]:
train_accuracy_scores = [] 
test_accuracy_scores = []

In [9]:
# Fitness determined by binary accuracy score
def fitness_func(solution, sol_idx):
    
    # Globals
    global nn_model, X_train, Y_train
    
    # Model instance and compile
    loss = tensorflow.keras.losses.BinaryCrossentropy()
    nn_model.compile(loss=loss, optimizer=Adam(), metrics=['accuracy'])
    
    # Fit
    history = nn_model.fit(X_train, Y_train, epochs=1)

    # Evaluate the fitness of the solution based on the final accuracy
    accuracy = history.history['accuracy'][-1]
    return accuracy

    

In [10]:
# Custom mutation function - exchanges one randomly chosen masked weight with a randomly chosen non-masked weight
def mutation_func(masked_population, nn_model):
    
    # Global variables
    global weights
    
    # Arrays of the indices for unmasked and maked weights
    unmasked_indices = np.where(masked_population == 1)[0]
    masked_indices = np.where(masked_population == 0)[0]
    
    # Choose random unmaked weight and masked weight
    if len(unmasked_indices) > 0 and len(masked_indices) > 0:
        rand_unmasked_idx = np.random.choice(unmasked_indices)
        rand_masked_idx = np.where(masked_population[rand_unmasked_idx] == 0)[0][0]
    
        # Save old value of unmasked weight
        old_value = weights[rand_unmasked_idx].copy()
    
        # Exchange values
        weights[rand_unmasked_idx] = weights[rand_masked_idx]
        weights[rand_masked_idx] = old_value
        nn_model.set_weights(weights)
        
        # Return updated network (along with it's weights)
        return nn_model

In [11]:
# On-generation function call

# Displays generation number, accuracy score, and fitness score
def callback_generation(ga_instance):   
    
    # Global
    global train_accuracy_scores, test_accuracy_scores
    
    # Print generation number and accuracy score
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Accuracy    = {accuracy}".format(accuracy=fitness_func(solution, sol_idx)))
    print("Fitness = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    
    
    # Train and test scores
    y_pred_tr = nn_model.evaluate(X_train, Y_train, verbose=0)
    train_accuracy = y_pred_tr[1]
    train_accuracy_scores.append(train_accuracy)
    
    y_pred_ts = nn_model.evaluate(X_test, Y_test, verbose=0)
    test_accuracy = y_pred_ts[1]
    test_accuracy_scores.append(test_accuracy)
    

In [12]:
# Make neural network model using Sequential keras model
# Define the architecture of your neural network
nn_model = Sequential()
nn_model.add(Dense(11, input_dim = len(X_train[0, :]), activation = 'sigmoid'))
nn_model.add(Dense(22, activation = 'sigmoid'))
nn_model.add(Dense(1, activation = 'sigmoid'))

Metal device set to: Apple M2


In [13]:
weights = np.concatenate([layer.flatten() for layer in nn_model.get_weights()]) #1-D array for connection weights
masked_population = np.zeros_like(weights) # Same size array filled with zeros for masking
num_unmasked = int(0.1 * weights.size) # 10% of weights unmasked
num_masked = weights.size - num_unmasked # Total number of masked weights
nonzero_indices = np.random.choice(weights.size, size=num_unmasked, replace=False) # Randomly select an unmasked index

# Randomly initialize a masked weights to an unmasked weight
masked_population[nonzero_indices] = np.random.choice(weights[nonzero_indices], size=num_unmasked)

In [14]:
# Make initial population based on network weights
keras_ga = pygad.kerasga.KerasGA(model=nn_model,
                                 num_solutions=20)

In [15]:
# Algorithm Parameters/Hyperparameters
# Default parent selection is generational in a PyGad GA
num_parents_mating = 2
num_generations = 10
keep_elitism = 2 # Weak elitism
crossover_type = "uniform" 
crossover_probability = 0.9
mutation_percent_genes= 10
initial_population = keras_ga.population_weights * 2

In [16]:
# Instantiate genetic algorithm with Parameters/Hyperparameters
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       initial_population= initial_population,
                       crossover_type= crossover_type,
                       crossover_probability= crossover_probability,
                       fitness_func=fitness_func,
                       mutation_percent_genes= mutation_percent_genes,
                       mutation_type=mutation_func, 
                       keep_elitism=keep_elitism,
                       on_generation=callback_generation)
solution, solution_fitness, sol_idx = ga_instance.best_solution()

2023-03-12 22:02:17.541612: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


35/35 [==============================] - 1s 5ms/step - loss: 0.5245 - accuracy: 0.7429


In [ ]:
ga_instance.run()

35/35 [==============================] - 1s 9ms/step - loss: 0.5119 - accuracy: 0.7527
Generation = 1
35/35 [==============================] - 1s 7ms/step - loss: 0.5122 - accuracy: 0.7518
Accuracy    = 0.7517856955528259
35/35 [==============================] - 1s 9ms/step - loss: 0.5092 - accuracy: 0.7482


35/35 [==============================] - 1s 12ms/step - loss: 0.5053 - accuracy: 0.7580
Fitness = 0.7580357193946838
35/35 [==============================] - 1s 9ms/step - loss: 0.5002 - accuracy: 0.7527
Generation = 2
35/35 [==============================] - 1s 9ms/step - loss: 0.4993 - accuracy: 0.7563
Accuracy    = 0.7562500238418579
35/35 [==============================] - 7s 135ms/step - loss: 0.4965 - accuracy: 0.7598


35/35 [==============================] - 1s 10ms/step - loss: 0.4940 - accuracy: 0.7616
Fitness = 0.7660714387893677
35/35 [==============================] - 1s 14ms/step - loss: 0.4878 - accuracy: 0.7652
Generation = 3
35/35 [==============================] - 1s 14ms/step - loss: 0.4869 - accuracy: 0.7661
Accuracy    = 0.7660714387893677
35/35 [==============================] - 2s 22ms/step - loss: 0.4807 - accuracy: 0.7732
Fitness = 0.7732142806053162
35/35 [==============================] - 2s 16ms/step - loss: 0.4804 - accuracy: 0.7688


35/35 [==============================] - 1s 14ms/step - loss: 0.4738 - accuracy: 0.7705
Generation = 4
35/35 [==============================] - 2s 17ms/step - loss: 0.4726 - accuracy: 0.7732
Accuracy    = 0.7732142806053162
35/35 [==============================] - 2s 17ms/step - loss: 0.4660 - accuracy: 0.7723
Fitness = 0.7758928537368774
35/35 [==============================] - 2s 18ms/step - loss: 0.4643 - accuracy: 0.7714


35/35 [==============================] - 2s 23ms/step - loss: 0.4585 - accuracy: 0.7741
Generation = 5
35/35 [==============================] - 2s 17ms/step - loss: 0.4585 - accuracy: 0.7768
Accuracy    = 0.7767857313156128
35/35 [==============================] - 2s 22ms/step - loss: 0.4514 - accuracy: 0.7911
Fitness = 0.7910714149475098
35/35 [==============================] - 2s 32ms/step - loss: 0.4451 - accuracy: 0.7857


35/35 [==============================] - 2s 24ms/step - loss: 0.4419 - accuracy: 0.7839
Generation = 6
35/35 [==============================] - 2s 23ms/step - loss: 0.4419 - accuracy: 0.7866
Accuracy    = 0.7866071462631226
35/35 [==============================] - 2s 15ms/step - loss: 0.4333 - accuracy: 0.7929
Fitness = 0.793749988079071
35/35 [==============================] - 3s 30ms/step - loss: 0.4251 - accuracy: 0.7982


35/35 [==============================] - 3s 32ms/step - loss: 0.4244 - accuracy: 0.7964
Generation = 7
35/35 [==============================] - 2s 36ms/step - loss: 0.4245 - accuracy: 0.7955
Accuracy    = 0.7955357432365417
35/35 [==============================] - 2s 39ms/step - loss: 0.4173 - accuracy: 0.8071
Fitness = 0.8071428537368774
35/35 [==============================] - 4s 39ms/step - loss: 0.4134 - accuracy: 0.8107


In [ ]:
solution, solution_fitness, sol_idx = ga_instance.best_solution()

In [ ]:
# Best solution display
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
print("Best fitness value reached after {best_solution_generation} generations.".format(best_solution_generation=ga_instance.best_solution_generation))

In [ ]:
best_solution_weights = pygad.kerasga.model_weights_as_matrix(model=nn_model, weights_vector=ga_instance.best_solution()[0])
nn_model.set_weights(best_solution_weights)

In [ ]:
# Plot fitness over generations
ga_instance.plot_fitness(title= "PyGad/Keras Generation vs. Fitness")

# Analysis of GA Instance

We can see the accuracy and fitness scores through each of our 10 generations above. No two accuracy scores or fitness scores are the same, but there is some random fluctuations of improvements and regressions in our scores. These discrepancies occur in such a manner because 10 generations is not enough time for this genetic algorithm to train our neural network. Running far more generations would increase our model fitness/accuracy because the algorithm would cover far more of the search space. It is a great sign that these values are changing though. This confirms that our model is undergoing learning, and the particular genetic algorithm instance is doing it's job.

I would guess that the high crossover probability paired with uniform crossover type scrambles and changes the weight solutions significantly after each generation, which could explain the limited improvement in our predictions as well.

Training this model also took far longer than the evolutionary strategy assignment. The population size is far larger and the weights undergo both crossover AND mutation compared to just adaptive mutation in our last assignment.

In [ ]:
# Use GA and custom weights to predict using our model
# Training predictions
training_predictions = pygad.kerasga.predict(model=nn_model,
                                    solution=solution,
                                    data=X_train)

# Testing predictions
testing_predictions = pygad.kerasga.predict(model=nn_model,
                                    solution=solution,
                                    data=X_test)

In [ ]:
test_loss, test_accuracy = nn_model.evaluate(X_test, Y_test)
print('Test accuracy:', test_accuracy)

In [ ]:
train_loss, train_accuracy = nn_model.evaluate(X_train, Y_train)
print('Train accuracy:', train_accuracy)

In [ ]:
# Make confusion matrices
confusion_matrix_test = confusion_matrix(Y_test, testing_predictions.round()) 
confusion_matrix_train = confusion_matrix(Y_train, training_predictions.round())

In [ ]:
# Visualize monfusion matrices using seaborn like other assignments
import seaborn as sns

# Test data
sns.heatmap(confusion_matrix_test, annot=True)
plt.title('Confusion Matrix Test Data')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()

# Train data
sns.heatmap(confusion_matrix_train, annot=True)
plt.title('Confusion Matrix Train Data')
plt.ylabel('Actual Values')
plt.xlabel('Predicted Values')
plt.show()

# Summary/Report

Genetic algorithms are ideal for complex optimization problems such as this neural network, binary classification problem. Some of the main takeaways and lessons I've learned from this assignment involve computational resoures and hyperparameter instantiation. It's extremely hard to visualize and choose what paramters and hyperparameters are necessary for maximum learning and optimization. Playing around with different values, selection types, population sizes, and mutations all have profound affects on our genetic algorithm which was visualized perfectly in this assignment. Another big takeaway for me was the difference in computation time. Training and running my neural network with my GA took substantially longer than the previous assignment we did with an ES. The population size and amount of operations that happen in this GA take a major toll on runtime and computational effort.

Visualizing the improvement through the genetic algorithm make it easy for me to see it's effectiveness. We can see this preformance through our confusion matrices and our generational output. Accuracy and fitness scores improved after every generation, and our confusion matrices have both improved over our last assignments corresponding matrices. 